In [ ]:
import requests, csv, os
from bs4 import BeautifulSoup
url = 'https://www.dtmodelmanagement.com/women/'
link='dtmodelmanagement.com'

In [ ]:
soup = BeautifulSoup(requests.get(url).text,'html5lib')
models = soup.find('div',attrs={'id':'navigation'}).findAll('div',attrs={'class':'listElement'})
model_names = [i.img['alt'] for i in models]
model_links = [f"{i.a['href']}stats" for i in models]
models_dict=dict(zip(model_names,model_links))
models_details=[]
print(f"{len(models_dict)} models")


In [ ]:
if not os.path.exists(link):
    os.mkdir(link)
for k,v in models_dict.items():
    os.makedirs(f"{link}/{k}", exist_ok=True)
    model_info={}
    model_info['NAME'] = k
    model_info['URL'] = v
    print(k,v)
    soup2 = BeautifulSoup(requests.get(v).text,'html5lib')
    for i in soup2.find('div',{'class':'measurement'}).findAll('li'):
        info = i.text.split(':')
        model_info[info[0]]=info[1].strip("'")
    for i in soup2.findAll('div',{'class':'picture-container'}):
        lnk = i.img['data-original']
        file_path =f"{link}/{k}/{lnk.split('/')[-1]}"
        if '?' in file_path:
            file_path = file_path.split('?')[0]
        if not os.path.exists(file_path):
            try:
                with open(file_path,'wb') as f:
                    f.write(requests.get(link).content)
            except:
                print("PROBABLE FILE SYSTEM ERROR ENCOUNTERED")
    models_details.append(model_info)
print(models_details)


In [ ]:

with open(f"./{link}/models_info.csv",'w') as f:
    c = csv.DictWriter(f, ['NAME','URL','HEIGHT','BUST','WAIST','HIPS','DRESS','SHOES','EYES','HAIR'])
    c.writeheader()
    for row in models_details:
        c.writerow(row)